# Código 08: Matrizes de translação, rotação e escala

## Código pré loop principal

### Inicialização do glfw e inicialização da janela

In [206]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np
import random

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

### Criando a janela

Aqui, vamos pegar informações do monitor atual. Em especial, o tamanho dele. Assim, conseguiremos fazer uma janela cheia.

In [207]:
# Obtenha a lista de monitores disponíveis
monitores = glfw.get_monitors()
# Escolha o monitor em que você deseja exibir a janela em tela cheia (por exemplo, o primeiro monitor)
monitor = monitores[0]
# Configure as dimensões da janela com base nas configurações do monitor
video_mode = glfw.get_video_mode(monitor)
WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
WIDTH_WINDOW : int = int(1*WIDTH_WINDOW)
HEIGHT_WINDOW : int = int(0.95*HEIGHT_WINDOW)
WIDTH_WINDOW : int = 900
HEIGHT_WINDOW : int = 500
#Exiba a tela
TITLE: str = "Transformação Geométrica 2D"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
#Definindo a posição da janela
POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

Criaremos um qualificador `uniform` para armazenar a matriz de transformação que será aplicado a todos os vértices. Também fazemos a matriz multiplicar a posição (vide teoria de matriz de transformação)

In [208]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Solicitando espaço, compilando e linkando

In [209]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

In [210]:
#Calculando aspect_radius
aspect_radius_x = 1
aspect_radius_y = 1

if WIDTH_WINDOW > HEIGHT_WINDOW: 
    aspect_radius_x = HEIGHT_WINDOW / WIDTH_WINDOW
else:
    aspect_radius_y = WIDTH_WINDOW / HEIGHT_WINDOW

#Criando a nave
nave_points =   [
                    ( 0.00, +0.00), 
                    (-0.06, -0.03), 
                    (-0.06, 0.03)
                ]



In [211]:
import math

#Pi: valor constante
PI : float = 3.14
#Número de vértices: define a qualidade do circulos
NUM_VERTEX : int = 32
#Raio da circunferencia
RAIUS : float = 0.03
#Variável auxiliar para calcular os vértices
temp_angle = 0.0
#Centro da circunferência
shot_x = -0.03
shot_y =  0.00
#Pontos do tiro
shot_points = []
for counter in range(NUM_VERTEX):
    #Variação do angulo em 32 vezes
    temp_angle += 2*PI/NUM_VERTEX 
    #Cálculos dos valores de (x,y)
    x = math.cos(temp_angle)*RAIUS*aspect_radius_x + shot_x
    y = math.sin(temp_angle)*RAIUS*aspect_radius_y + shot_y
    shot_points += [(x,y)]

In [212]:
#Criando espaço
TOTAL_LEN : int = len(nave_points) + len(shot_points)
vertices = np.zeros(TOTAL_LEN, [("position", np.float32, 2)])
vertices["position"] = nave_points + shot_points

### Manipulação dos espaços de dados

In [213]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_position = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_position)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_position, 2, GL_FLOAT, False, stride, offset)

### Capturando localização do qualificador `uniform` responsável pela cor e pela matriz de transformação

In [214]:
loc_color = glGetUniformLocation(program, "color")
loc_mat = glGetUniformLocation(program, "mat_transformation")

### Eventos de teclado e mouse

In [215]:
# Coordenadas da nave
nave_x = nave_y = 0

# Coordenadas do tiro
shot_x = shot_y = 0

# Cores da nave
R_NAVE = random.uniform(0.5,1.0)
G_NAVE = random.uniform(0.5,1.0)
B_NAVE = random.uniform(0.5,1.0)

# Cores do tiro
R_SHOT = random.uniform(0.5,1.0)
G_SHOT = random.uniform(0.5,1.0)
B_SHOT = random.uniform(0.5,1.0)

# Escala da nave
scale_nave = 1.0

# Rotação da nave
angle_nave = 0.0

# Angulo do tiro
angle_shot = 0.0

# Existe tiro
flag_shot = False

# Determina sentido de movimentação do tiro
incr_shot_x = incr_shot_y = 0.0001

# Coordenadas do cursor
cursor_x = cursor_y = 0

def key_event(window,key,scancode,action,mods):
    # Global: variáveis não estão sendo criadas como locais dentro da função
        # mas sim referenciando as variáveis globais definidas fora da função.
    global nave_x, nave_y, scale_nave, angle_nave, R_NAVE, G_NAVE, B_NAVE, flag_shot, incr_shot_x, incr_shot_y, shot_x, shot_y, angle_shot
    
    linear_speed = 0.05
    if key == 87: nave_y += linear_speed #Tecla W
    if key == 83: nave_y -= linear_speed #Tecla S
    if key == 65: nave_x -= linear_speed #Tecla A
    if key == 68: nave_x += linear_speed #Tecla D
    
    scale_nave_speed = 0.2
    if key == 81: scale_nave += scale_nave_speed #Tecla Q
    if key == 69: scale_nave -= scale_nave_speed #Tecla E
    
    angle_nave_speed = 0.139 #radianos
    if key == 79: angle_nave += angle_nave_speed #Tecla O
    if key == 80: angle_nave -= angle_nave_speed #Tecla P

    color_speed = 0.1
    if mods == 1:
        if key == 82: R_NAVE += color_speed #Tecla R
        if key == 71: G_NAVE += color_speed #Tecla G
        if key == 66: B_NAVE += color_speed #Tecla B
    else:
        if key == 82: R_NAVE -= color_speed #Tecla R
        if key == 71: G_NAVE -= color_speed #Tecla G
        if key == 66: B_NAVE -= color_speed #Tecla B

    if key == 32 :
        flag_shot = True
        shot_x = nave_x
        shot_y = shot_y
        angle_shot = angle_nave

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    
glfw.set_key_callback(window, key_event)

def mouse_event(window,button,action,mods):
    global nave_x, nave_y, cursor_x, cursor_y
    if button == 1: 
        nave_x = cursor_x
        nave_y = cursor_y
    # print('[mouse event] button=',button)
    # print('[mouse event] action=',action)
    # print('[mouse event] mods=',mods)
    # print('-------')

glfw.set_mouse_button_callback(window, mouse_event)

def cursor_event(window, xpos, ypos):
    global cursor_x, cursor_y
    cursor_x = xpos*(2/WIDTH_WINDOW) - 1
    cursor_y = 1 - ypos*(2/HEIGHT_WINDOW)
    #print(f"Posição do cursor: ({cursor_x}, {cursor_y})")

glfw.set_cursor_pos_callback(window, cursor_event)


### Exibindo na tela

In [216]:
glfw.show_window(window)

## Loop principal

* `glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_transformation)`: função usada para alterar o conteúdo do qualificador `uniform` com localização `loc_mat`.

In [217]:
def calc_mat_transform(a,b,c):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = c.reshape(4,4)
    return np.dot(np.dot(m_a, m_b), m_c)

def calc_angle_direction(cursor_x, cursor_y, nave_x, nave_y):
    # Calcular a direção do objeto para o cursor
    direction = np.array([cursor_x, cursor_y]) - np.array([nave_x, nave_y])
    return np.arctan2(direction[1], direction[0])

def calc_limits_coord_nave(nave_x, nave_y, borda_esq = -1, borda_dir = 1, borda_sup = 1, borda_inf = -1):    
    if nave_y >= borda_sup:
        nave_y = borda_sup
    elif nave_y <= borda_inf:
        nave_y = borda_inf

    if nave_x >= borda_dir:
        nave_x = borda_dir
    elif nave_x <= borda_esq:
        nave_x = borda_esq
    
    return nave_x, nave_y

def calc_limits_coord_shot(shot_x, shot_y, flag_shot, nave_x, nave_y, borda_esq = -1, borda_dir = 1, borda_sup = 1, borda_inf = -1):
    if shot_x >= borda_dir or shot_x <= borda_esq or shot_y >= borda_sup or shot_y <= borda_inf:
        shot_x = nave_x
        shot_y = nave_y
        flag_shot = False
    
    return shot_x, shot_y, flag_shot

In [218]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    # glPolygonMode(GL_FRONT,GL_LINE)
    # glPolygonMode(GL_BACK,GL_LINE)
    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Recalcula nave_x e nave_y caso ultrapassem a borda
    nave_x, nave_y = calc_limits_coord_nave(nave_x,nave_y)
    shot_x, shot_y, flag_shot = calc_limits_coord_shot(shot_x, shot_y, flag_shot, nave_x, nave_y)

    # > Matriz de rotacao
    angle_nave = calc_angle_direction(cursor_x, cursor_y, nave_x, nave_y)
    cos = np.cos(angle_nave)
    sin = np.sin(angle_nave)
    mat_rotation_nave = np.array([  cos, -sin, 0.0, 0.0, 
                                    sin,  cos, 0.0, 0.0, 
                                    0.0,  0.0, 1.0, 0.0, 
                                    0.0,  0.0, 0.0, 1.0], np.float32)
    
    # > Matriz de escala
    # Declara variável scale_nave_x e scale_nave_y para serem aplicados a cada eixo
    # Como queremos uma variação igual de escala no dos eixos, eles são iguais
    scale_nave_x = scale_nave_y = scale_nave
    mat_scale_nave =    np.array([  scale_nave_x,     0.0, 0.0, 0.0, 
                                   0.0, scale_nave_y, 0.0, 0.0, 
                                   0.0,     0.0, 1.0, 0.0, 
                                   0.0,     0.0, 0.0, 1.0], np.float32)
    
    # > Matriz de translação
    mat_translation_nave = np.array([   1.0, 0.0, 0.0, nave_x, 
                                        0.0, 1.0, 0.0, nave_y, 
                                        0.0, 0.0, 1.0, 0.0, 
                                        0.0, 0.0, 0.0, 1.0], np.float32)
    
    #> Cálculo da matriz de transformação
    #É necessário colocar a matriz de rotação mais afastado do ponto na hora do cálculo
    mat_transformation_nave = calc_mat_transform(mat_translation_nave, mat_rotation_nave, mat_scale_nave)
    
    #Aplica transformações, cores e rasteriza para a nave
    glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_transformation_nave)
    glUniform4f(loc_color, R_NAVE, G_NAVE, B_NAVE, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, 0, len(nave_points))

    #Aplica transformações, cores e rasteriza para o tiro
    if flag_shot :
        #shot_x += incr_shot_x
        shot_y += incr_shot_y
        print(f"Valor do angulo da nave e{angle_nave} e do tiro é {angle_shot}")
        cos = np.cos(angle_shot)
        sin = np.sin(angle_shot)
        mat_rotation_shot = np.array([      cos, -sin, 0.0, 0.0, 
                                            sin,  cos, 0.0, 0.0, 
                                            0.0,  0.0, 1.0, 0.0, 
                                            0.0,  0.0, 0.0, 1.0], np.float32)
    
        mat_translation_shot = np.array([   1.0, 0.0, 0.0, shot_x, 
                                            0.0, 1.0, 0.0, shot_y, 
                                            0.0, 0.0, 1.0, 0.0, 
                                            0.0, 0.0, 0.0, 1.0], np.float32)
        
        mat_transformation_shot = calc_mat_transform(mat_translation_shot, mat_rotation_shot, np.identity(4))
        glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_translation_shot)
    #Provavelmente não fica certinho porque
        #1. Quando eu nao estou atirando, o tiro segue a nave certinho porque estou usando a matriz de transformação da nave no tiro também
        #2. Para corrigir, vou ter que salvar a matriz de rotação da nave e aplicar no tiro antes de aplica a transformação de deslocação
        #3. Vou fazer isso, mas agora só quando estiver trabalhando com orientação a objeto
    glDrawArrays(GL_TRIANGLE_FAN, len(nave_points), len(shot_points))

    glfw.swap_buffers(window)

glfw.terminate()

Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e-2.718738727456852 e do tiro é -2.718738727456852
Valor do angulo da nave e